In [7]:
import requests
from bs4 import BeautifulSoup
import os
import numpy as np

In [ ]:
def _retrieve_electron_binding_energies():
    # URL of the website
    url = "https://physics.nist.gov/cgi-bin/ASD/ie.pl"
    
    # Parameters for the POST request
    data = {
        'spectra': 'H-Og',
        'submit': 'Retrieve Data',
        'units': '1',
        'format': '0',
        'order': '0',
        'at_num_out': 'on',
        'ion_charge_out': 'on',
        'e_out': '1',
        'unc_out': 'on'
    }
    
    # Send a POST request to the website
    response = requests.post(url, data=data)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Use BeautifulSoup to parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find the table in the HTML
        table = soup.find('table', {'border': '1'})
        
        if table:
            # Initialize the dictionary
            data_dict = {}
    
            # Iterate over the rows of the table, skipping the header row
            for row in table.find_all('tr')[1:]:
                cells = row.find_all('td')
                if len(cells) == 4:
                    at_num = cells[0].text.strip()
                    ion_charge = cells[1].text.strip()
                    binding_energy = cells[2].text.strip()
                    uncertainty = cells[3].text.strip()
    
                    # Create a key-value pair in the dictionary
                    if at_num not in data_dict:
                        data_dict[at_num] = []
                    data_dict[at_num].append({
                        'Ion Charge': ion_charge,
                        'Binding Energy (eV)': binding_energy,
                        'Uncertainty (eV)': uncertainty
                    })
    
            # Print the dictionary
            for key, values in data_dict.items():
                print(f"At. Num. {key}:")
                for value in values:
                    print(f"  {value}")
        else:
            print("Table not found in the HTML content.")
    else:
        print("Failed to retrieve data from the website.")

In [4]:
def _save_electron_binding_energies(data_dict, filename = 'electron_binding_energies'):
    npz_data = {key: np.array(value) for key, value in data_dict.items()}
    # Save the data to a .npz file
    np.savez(filename, **npz_data)

In [38]:
def _load_electron_binding_energies(filename = 'electron_binding_energies.npz'):
    data = np.load(filename, allow_pickle=True)
    # Convert the loaded data back to a dictionary format
    return {key: data[key].tolist() for key in data}

In [45]:
def _convert_binding_energy_to_float(s):
    # Remove square brackets and parentheses
    s = s.strip('[]()')
    s = s.replace(' ', '')
    return float(s)

In [8]:
_save_electron_binding_energies(data_dict)

In [39]:
a = _load_electron_binding_energies()

In [44]:
_convert_binding_energy_to_float(a['32'][0]['Binding Energy (eV)']),float(a['32'][0]['Uncertainty (eV)'])

(57080.0, 50.0)

In [33]:
data_dict

{'1': [{'Ion Charge': '0',
   'Binding Energy (eV)': '(13.598434599702)',
   'Uncertainty (eV)': '0.000000000012'}],
 '2': [{'Ion Charge': '0',
   'Binding Energy (eV)': '[79.005154539]',
   'Uncertainty (eV)': '0.000000025'},
  {'Ion Charge': '+1',
   'Binding Energy (eV)': '(54.4177655282)',
   'Uncertainty (eV)': '0.0000000010'}],
 '3': [{'Ion Charge': '0',
   'Binding Energy (eV)': '[203.4861711]',
   'Uncertainty (eV)': '0.0000012'},
  {'Ion Charge': '+1',
   'Binding Energy (eV)': '(198.0944562)',
   'Uncertainty (eV)': '0.0000012'},
  {'Ion Charge': '+2',
   'Binding Energy (eV)': '(122.45435913)',
   'Uncertainty (eV)': '0.00000011'}],
 '4': [{'Ion Charge': '0',
   'Binding Energy (eV)': '[399.14864]',
   'Uncertainty (eV)': '0.00005'},
  {'Ion Charge': '+1',
   'Binding Energy (eV)': '[389.82595]',
   'Uncertainty (eV)': '0.00005'},
  {'Ion Charge': '+2',
   'Binding Energy (eV)': '(371.614789)',
   'Uncertainty (eV)': '0.000004'},
  {'Ion Charge': '+3',
   'Binding Energy (eV